In [3]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from itertools import combinations 
from numpy import savetxt
import random

2022-10-09 22:36:42.090793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-09 22:36:42.090818: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras

In [11]:
input_df = pd.read_csv("data/input_data_nn_138.csv", sep=";")
output_df = pd.read_csv("data/output_data_nn_138.csv", sep=";")
df = pd.concat([input_df, output_df], axis=1)
df = df.loc[~(df==0).all(axis=1)].reset_index()
#print(output_df.describe().loc[['mean', 'std', 'min', 'max']].transpose().to_latex())

In [12]:
def augment(x, y, augmenter = None):
    x, augmenter = _add_interactions(x, augmenter)
    return x, y, augmenter, x.columns

def _add_interactions(df, augmenter):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]

    if augmenter:
        df = augmenter.transform(df)
    else:
        augmenter = PolynomialFeatures(interaction_only=True, include_bias=False)
        df = augmenter.fit_transform(df)

    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df, augmenter

In [13]:
def run_main(df, idx, idx2, ids_to_pred):    
    cols = input_df.columns.tolist() 
    new_df = df.copy()
    cols_output = [
        'tensao_barramento11', 'tensao_barramento12', 'tensao_barramento24',
        'pot_ativa_inj_barramento11', 'pot_ativa_inj_barramento12', 'pot_ativa_inj_barramento24',
        'pot_reativa_inj_barramento11', 'pot_reativa_inj_barramento12', 'pot_reativa_inj_barramento24'
    ]
    predicted_df = pd.DataFrame(columns=cols_output, index=ids_to_pred)
    
    cols_pred = ['tensao_barramento11']
    model_id = 'tensao_11'
    tensao_11_df, predicted_tensao_11 = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    predicted_df[cols_pred] = predicted_tensao_11
    
    cols_pred = ['tensao_barramento12']
    model_id = 'tensao_12'
    tensao_12_df, predicted_tensao_12 = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    predicted_df[cols_pred] = predicted_tensao_12  
        
    cols_pred = ['tensao_barramento24']
    model_id = 'tensao_24'
    tensao_24_df, predicted_tensao_24 = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    predicted_df[cols_pred] = predicted_tensao_24
    
    cols_pred = [
        'pot_ativa_inj_barramento11'
    ]
    model_id = 'ativa_11'
    ativa_df, predicted_ativa = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    predicted_df[cols_pred] = predicted_ativa
    
    cols_pred = [
        'pot_ativa_inj_barramento12'
    ]
    model_id = 'ativa_12'
    ativa_df, predicted_ativa = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    predicted_df[cols_pred] = predicted_ativa

    cols_pred = [
        'pot_ativa_inj_barramento24'
    ]
    model_id = 'ativa_24'
    ativa_df, predicted_ativa = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    predicted_df[cols_pred] = predicted_ativa
    
    
    cols_pred = [
        'pot_reativa_inj_barramento11'
    ]
    model_id = 'reativa_11'
    ativa_df, predicted_ativa = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    predicted_df[cols_pred] = predicted_ativa
    
    cols_pred = [
        'pot_reativa_inj_barramento12'
    ]
    model_id = 'reativa_12'
    ativa_df, predicted_ativa = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    predicted_df[cols_pred] = predicted_ativa

    cols_pred = [
        'pot_reativa_inj_barramento24'
    ]
    model_id = 'reativa_24'
    ativa_df, predicted_ativa = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    predicted_df[cols_pred] = predicted_ativa
    
    
    predicted_df.to_csv(f'results-138/{idx}-{idx2}.csv')
    
    return new_df

In [14]:
def run(df, cols, cols_pred, model_id, idx, ids_to_pred, update_test = False):    
    _df = df[cols + cols_pred]
    
    X, Y = _df[cols], _df[cols_pred]
    #X, Y, _, x_cols, = augment(X, Y)
    _df_augm = pd.concat([X, Y],  axis=1)

    df_for_testing = _df_augm.loc[ids_to_pred]
    df_for_training = _df_augm.loc[~_df_augm.index.isin(ids_to_pred)]
    
    df_for_training = df_for_training.sample(frac=1)
    
    trainX, trainY = df_for_training.drop(cols_pred, axis=1), df_for_training[cols_pred]
    testX, testY = df_for_testing.drop(cols_pred, axis=1), df_for_testing[cols_pred]
    
    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.transform(testX)
    
    print(f" ========== MODEL {model_id} {idx} ============")
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
    mc = ModelCheckpoint(f"models-new/ann-{model_id}-{idx}.h5", monitor='val_loss', mode='min', save_best_only=True, verbose=0)

    model = Sequential()
    model.add(Input(shape=(trainX.shape[1],)))
    model.add(Dense(trainX.shape[-1], activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(trainX.shape[-1], activation = 'relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(len(cols_pred)))

    model.compile(loss = 'mae', optimizer = 'adam')
    model.fit(
        trainX, trainY,
        batch_size = 30, epochs = 20,
        callbacks = [es, mc],
        validation_data = (testX, testY),
        verbose=1
    )

    _predictions = model.predict(testX) 
    
    rmse = sqrt(mean_squared_error(testY, _predictions))
    print(f"Col: {cols_pred[i]} - RMSE: {rmse}")
    results.append(rmse)
    
#     if update_test:
#         df_for_testing[cols_pred] = _predictions

    return pd.concat([df_for_training, df_for_testing]), _predictions

In [ ]:
indexes = np.array(df.index.tolist())
random.shuffle(indexes)
ids_to_pred = np.split(indexes, 3)

results_all = []
for i in range(4):
    results_per_ids = []
    for idx2 in range(len(ids_to_pred)):
        results = []
        run_main(df, i, idx2, ids_to_pred[idx2])
        results_per_ids.append(results)
        
    results_all.append(results_per_ids)

In [21]:
for _result in results_all:
    print(results_all.index(_result))
    for fold in _result:
        print(f"{_result.index(fold)}: {sum(fold)}")

0
0: 32.33497086128333
1: 36.889005154036376
2: 36.68749314981343
3: 31.932388827029182
1
0: 32.26412368037137
1: 31.990872924834257
2: 36.0298650015969
3: 32.30431549681663
2
0: 31.72844917901247
1: 29.84034210963254
2: 34.37873809844469
3: 38.184718100379435
3
0: 34.15181711482428
1: 30.558933149246954
2: 30.1250874789621
3: 31.1846092629998
4
0: 34.86386086977219
1: 28.93438134306144
2: 36.44611309999486
3: 33.4754096497843
5
0: 37.116041669193386
1: 37.296140359486756
2: 30.919280629845613
3: 39.44656374003831
6
0: 31.38229179727558
1: 30.653259296110438
2: 31.444674673125057
3: 36.76260480496647
7
0: 28.349547201857266
1: 33.73417886152881
2: 39.737148167795624
3: 37.59890026817541
8
0: 29.559890665146096
1: 40.83094120187875
2: 26.866222288396877
3: 37.813916555230335
9
0: 38.21746676037965
1: 37.003628372655385
2: 47.06004230959253
3: 34.13258223262406


In [26]:
print(f"Fold 0: {results_all[7][0]}")
print(f"Fold 1: {results_all[4][1]}")
print(f"Fold 2: {results_all[8][2]}")
print(f"Fold 3: {results_all[3][3]}")

Fold 0: [0.005618806104382909, 0.006374496128408189, 0.005003273984089787, 3.4735610445018055, 5.057355698378199, 3.6107973146928116, 7.466659649968084, 6.293739685672916, 2.4304372324265673]
Fold 1: [0.004912833436191945, 0.006315874222206748, 0.0050400543592956865, 5.882427740243566, 3.5072674248462787, 3.786373119903367, 6.751693446504084, 6.168161926473203, 2.822188923073245]
Fold 2: [0.004500589408310584, 0.004620877592182861, 0.005015842571451259, 3.953451113791048, 4.887246335988537, 4.617532544758241, 5.595597117037946, 5.241330146546544, 2.5569277207026158]
Fold 3: [0.005904257261356303, 0.004717165985727671, 0.005687405161361025, 3.2969980512393264, 5.985023419883928, 3.5217441168282497, 8.310837484354026, 6.692027891640731, 3.3616694706450936]


In [40]:
print(f"RMSE tensao_barramento11: {sqrt(mean_squared_error(df['tensao_barramento11'], df_best_predicted['tensao_barramento11']))}")
print(f"RMSE tensao_barramento12: {sqrt(mean_squared_error(df['tensao_barramento12'], df_best_predicted['tensao_barramento12']))}")
print(f"RMSE tensao_barramento24: {sqrt(mean_squared_error(df['tensao_barramento24'], df_best_predicted['tensao_barramento24']))}")

print(f"RMSE pot_ativa_inj_barramento11: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento11'], df_best_predicted['pot_ativa_inj_barramento11']))}")
print(f"RMSE pot_ativa_inj_barramento12: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento12'], df_best_predicted['pot_ativa_inj_barramento12']))}")
print(f"RMSE pot_ativa_inj_barramento24: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento24'], df_best_predicted['pot_ativa_inj_barramento24']))}")


RMSE tensao_barramento11: 0.005263612667191903
RMSE tensao_barramento12: 0.005570651504552013
RMSE tensao_barramento24: 0.005194712520332189
RMSE pot_ativa_inj_barramento11: 0.0
RMSE pot_ativa_inj_barramento12: 0.0
RMSE pot_ativa_inj_barramento24: 0.0
